In [1]:
%load_ext literary.module

# Package Generation

In [1]:
import shutil
from pathlib import Path

import nbformat
from nbconvert import Exporter
from traitlets import Bool, Instance, List, Type, Unicode, default
from traitlets.config import Config, Configurable

from ..transpile.exporter import LiteraryExporter
from .app import LiteraryApp

In [2]:
DEFAULT_IGNORE_PATTERNS = (".ipynb_checkpoints", "__pycache__", ".*")

Here we implement a `PackageBuilder` class. Because it operators upon the current project, it is a `ProjectOperator` subclass. We define a few useful configuration traits, such as the exporter and the output directory.

In [3]:
class LiteraryBuildApp(LiteraryApp):
    """Project operator which builds a Literary package from a set of notebook directories."""

    description = "Build a pure-Python package from a set of Jupyter notebooks"

    exporter = Instance(Exporter)
    exporter_class = Type(LiteraryExporter).tag(config=True)

    generated_dir = Unicode(
        "lib", help="Path to generated packages top-level directory"
    ).tag(config=True)

    ignore_patterns = List(
        Unicode(), help="List of patterns to ignore from source tree"
    ).tag(config=True)

    clear_generated = Bool(
        False,
        help="Clear generated directory before building, otherwise raise an Exception if non-empty.",
    ).tag(config=True)

    aliases = {
        **LiteraryApp.aliases,
        "ignore": "LiteraryBuildApp.ignore_patterns",
        "output": "LiteraryBuildApp.generated_dir",
        "packages": "LiteraryBuildApp.packages_dir",
    }
    flags = {
        "clear": (
            {"LiteraryBuildApp": {"clear_generated": True}},
            "Clear generated directory before building.",
        )
    }

Let's implement a lazy exporter getter

In [4]:
@patch(LiteraryBuildApp)
@default("exporter")
def _exporter_default(self):
    return self.exporter_class(parent=self, config=self.config)

The `generated_dir` directory should be resolved against the project path, so let's implement a getter.

In [5]:
@patch(LiteraryBuildApp)
@property
def generated_path(self) -> Path:
    return self.resolve_path(self.generated_dir)

By default, we want to ignore a number of different glob patterns to avoid bundling cache files etc.

In [6]:
@patch(LiteraryBuildApp)
@default("ignore_patterns")
def _ignore_patterns_default(self):
    return list(DEFAULT_IGNORE_PATTERNS)

We will build the package by recursively visiting all members of the source file system. Once we hit a notebook, we export it, whilst any non-ignored files are copied.

In [7]:
@patch(LiteraryBuildApp)
def _build_package_component(
    self,
    source_dir_path: Path,
    dest_dir_path: Path,
):
    """Recursively build a pure-Python package from a source tree

    :param source_dir_path: path to current source directory
    :param dest_dir_path: path to current destination directory
    :return:
    """
    # Ensure we have a destination
    dest_dir_path.mkdir(parents=True, exist_ok=True)

    for path in source_dir_path.iterdir():
        # Ignore any unwanted files or directories
        if any(path.match(p) for p in self.ignore_patterns):
            continue

        # Do not visit lib, ever!
        if path == self.generated_dir:
            continue

        # Find equivalent path in generated package
        relative_path = path.relative_to(source_dir_path)
        mirror_path = dest_dir_path / relative_path

        # Rewrite notebook in target directory
        if path.match("*.ipynb"):
            source, _ = self.exporter.from_notebook_node(
                nbformat.read(path, as_version=nbformat.NO_CONVERT)
            )
            mirror_path.with_suffix(".py").write_text(source)

        # Recurse into directory
        elif path.is_dir():
            self._build_package_component(path, mirror_path)

        # Copy file directly
        else:
            mirror_path.write_bytes(path.read_bytes())

Within the `packages_path`, we look for files and directories which become our generated set of packages. We call `build_package_component` for each found package.

In [8]:
@patch(LiteraryBuildApp)
def _build_packages(self):
    """Build the packages contained in `packages_path`."""
    self._build_package_component(self.packages_path, self.generated_path)

Before building, we need to clear the generated packages directory. Otherwise, renamed files would persist in the generation directory.

In [9]:
@patch(LiteraryBuildApp)
def _clear_generated_path(self):
    """Clear the contents of `generated_path`."""
    for p in self.generated_path.iterdir():
        if not self.clear_generated:
            raise ValueError(
                "Generated directory is not empty, and `clear_generated` is not set."
            )

        if p.is_file():
            p.unlink()

        else:
            shutil.rmtree(p)

To perform a build, we simply call the aforementioned functions to clear the generated directory and build the list of packages.

In [10]:
@patch(LiteraryBuildApp)
def start(self):
    """Build a pure-Python package from a literary source tree."""
    self.generated_path.mkdir(parents=True, exist_ok=True)

    # Empty destination contents
    self._clear_generated_path()

    # We need a source package
    if not self.packages_path.exists():
        raise FileNotFoundError(f"Source path {self.packages_path!r} does not exist")

    self._build_packages()